# Run the three-layer IRM

This notebook will demonstrate the inclusion of ocean heat uptake.

We will use example forcing from RFMIP to demonstrate.

Outstanding questions here is the emulated temperature so bad in some models, e.g. IPSL? Lawrence and Hege-Beate also find this, so possibly not a weakness of this particular model.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import scipy.linalg
import scipy.stats
from scipy.interpolate import interp1d
import json
from tqdm import tqdm

from fair.earth_params import seconds_per_year, earth_radius
from fair.energy_balance_model import EnergyBalanceModel

In [ ]:
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins_ebm3.csv")
)

In [ ]:
models = df['model'].unique()
models

In [ ]:
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]

In [ ]:
time = np.arange(1850, 2101)
time.shape

In [ ]:
df_forcing = pd.read_csv(
    os.path.join("..", "data", "forcing", "RFMIP-ERF-tier2.csv")
)

In [ ]:
ebm = EnergyBalanceModel(**params['GISS-E2-1-G']['r1i1p1f1'])

In [ ]:
ebm.add_forcing(forcing = df_forcing['GISS-E2-1-G TOT'].values, timestep=1)

In [ ]:
ebm.run()

In [ ]:
ebm.temperature

In [ ]:
time = np.arange(1850.5, 2101)

In [ ]:
pl.plot(time, ebm.temperature[:,0], label='surface / top ocean layer')
pl.plot(time, ebm.temperature[:,1], label='second ocean layer')
pl.plot(time, ebm.temperature[:,2], label='deep ocean layer')
pl.ylabel('K relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 temperature change')
pl.legend()

In [ ]:
pl.plot(time, ebm.ocean_heat_content_change/10**21)
pl.ylabel('ZJ relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 change in OHC')

In [ ]:
# A very, very crude estimate of global thermosteric sea level rise
# use Kuhlbrodt & Gregory 2012 multi-model-mean of 0.113 mm / ZJ thermal expansion coefficient

pl.plot(time, ebm.ocean_heat_content_change/10**21*0.113)
pl.ylabel('mm relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 thermosteric sea level rise')

In [ ]:
pl.plot(time, ebm.toa_imbalance)
pl.ylabel('W m$^{-2}$ relative to 1850')
pl.title('GISS-E2-1-G SSP2-4.5 TOA energy imbalance')

## Test with non-yearly timestep

For this I'll use the IPCC AR6 forcing, but interpolated to monthly and monthly volcanic forcing substituted in.

Use Donald Cummins' default parameters for the EBM.

In [ ]:
df_volcanic = pd.read_csv(
    os.path.join("..", "data", "forcing", "volcanic_sAOD_monthly_175001-201912.csv"),
    index_col=0
)
df_volcanic

In [ ]:
# see https://github.com/chrisroadmap/ar6/blob/main/notebooks/040_chapter2_volcanic_erf.ipynb
df_volcanic['forcing'] = df_volcanic['stratospheric_AOD'] * (-20) + 0.2582047762183514
pl.plot(df_volcanic['forcing'])

In [ ]:
df_solar_anthro = pd.read_csv(
    os.path.join("..", "data", "forcing", "AR6_ERF_1750-2019_solar_anthro.csv"),
    index_col=0
)
df_solar_anthro

In [ ]:
f = interp1d(np.arange(1750.5, 2020), df_solar_anthro['solar'], fill_value='extrapolate')
solar = f(np.arange(1750+1/24, 2020, 1/12))
f = interp1d(np.arange(1750.5, 2020), df_solar_anthro['total_anthropogenic'], fill_value='extrapolate')
anthro = f(np.arange(1750+1/24, 2020, 1/12))
ar6_forcing_monthly = solar + anthro + df_volcanic['forcing'].values
pl.plot(np.arange(1750+1/24, 2020, 1/12), ar6_forcing_monthly)

In [ ]:
# run: heat capacities must be adjusted for new timestep
#params_giss_monthly = params['GISS-E2-1-G']['r1i1p1f1']

ebm = EnergyBalanceModel(timestep=1/12, **params['GISS-E2-1-G']['r1i1p1f1'])
ebm.add_forcing(forcing=ar6_forcing_monthly, timestep=1/12)
ebm.run()

In [ ]:
time=np.arange(1750+1/24, 2020, 1/12)

In [ ]:
pl.plot(time, ebm.temperature[:,0], label='surface / top ocean layer')
pl.plot(time, ebm.temperature[:,1], label='second ocean layer')
pl.plot(time, ebm.temperature[:,2], label='deep ocean layer')
pl.ylabel('K relative to 1750')
pl.title('Forcing-driven historical temperature change')
pl.legend()

In [ ]:
pl.plot(time, ebm.ocean_heat_content_change/10**21)
pl.ylabel('ZJ relative to 1750')
pl.title('Forcing-driven change in OHC')

In [ ]:
# A very, very crude estimate of global thermosteric sea level rise
# use Kuhlbrodt & Gregory 2012 multi-model-mean of 0.113 mm / ZJ thermal expansion coefficient

pl.plot(time, ebm.ocean_heat_content_change/10**21*0.113)
pl.ylabel('mm relative to 1750')
pl.title('Forcing-driven thermosteric sea level rise')

In [ ]:
pl.plot(time, ebm.toa_imbalance)
pl.ylabel('W m$^{-2}$ relative to 1850')
pl.title('Forcing-driven TOA energy imbalance')

## Run RFMIP with their tuned scenarios

In [ ]:
results = {}
rfmip_forcing = {}
models = ['CanESM5', 'CNRM-CM6-1', 'GFDL-CM4', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'NorESM2-LM']
runs = {}
for model in models:
    runs[model] = 'r1i1p1f1'
# exceptions
runs['CanESM5'] = 'r1i1p2f1'
runs['CNRM-CM6-1'] = 'r1i1p1f2'
runs['HadGEM3-GC31-LL'] = 'r1i1p1f3'

In [ ]:
for model in models:
    rfmip_forcing[model] = df_forcing['%s TOT' % model].values
    ebm = EnergyBalanceModel(**params[model][runs[model]])
    ebm.emergent_parameters()
    ebm.add_forcing(forcing = rfmip_forcing[model], timestep=1)
    ebm.run()
    
    # TODO: method to easily save out results to an object or dict. Glen Harris's version had a nice Results class for this
    results[model] = ebm.__dict__

In [ ]:
# this is what the combined input/output dict looks like
results[model]

In [ ]:
for model in models:
    pl.plot(np.arange(1850, 2101), results[model]['temperature'][:,0], label=model)
pl.legend()

In [ ]:
for model in models:
    print(model, results[model]['ecs'], results[model]['tcr'])

### Compare to each model's CMIP6 ScenarioMIP SSP2-4.5

In [ ]:
df = pd.read_csv(os.path.join("..", "data", "cmip6-hbf", "ssp245.csv"))
df
# I only want certain i, p, f combinations of some models
filters = {
    'CanESM5': 'i1p2f1',
    'CNRM-CM6-1': 'i1p1f2',
    'GFDL-CM4': 'i1p1f1',
    'GISS-E2-1-G': 'i1p1f2',
    'HadGEM3-GC31-LL': 'i1p1f3',
    'IPSL-CM6A-LR': 'i1p1f1',
    'MIROC6': 'i1p1f1',
    'NorESM2-LM': 'i1p1f1',
}

cmip6 = {}

for model in models:
    condition = (df['climate_model']==model) & df['member_id'].str.contains(filters[model]) & (df['variable']=='tas')
    cmip6[model] = {}
    cmip6[model]['tas'] = df.loc[condition].values[:,9:]
    condition = (df['climate_model']==model) & df['member_id'].str.contains(filters[model]) & (df['variable']=='rndt')
    cmip6[model]['rndt'] = df.loc[condition].values[:,9:]

In [ ]:
fig, ax = pl.subplots(2,4, figsize=(16, 9))
for i, model in enumerate(models):
    ax[i%2,i//2].plot(np.arange(1850, 2101), results[model]['temperature'][:,0], color='k', label='Emulation')
    #ax[0,0].fill_between(np.arange(1850, 2101), np.min(cmip6[model]['tas'], axis=0), np.max(cmip6[model]['tas'], axis=0), color='r', alpha=0.3)
    ax[i%2,i//2].plot(np.arange(1850, 2101), cmip6[model]['tas'].mean(axis=0), color='r', label='CMIP6 (single-model ensemble mean)')
    ax[i%2,i//2].set_title(model)
ax[0,0].legend()

In [ ]:
# I want to see what an ensemble looks like!
# wonder why this takes a long time compared to FaIR... probably not a lot of parallelisation?
n_ens = 100

ens = {}
for im, model in tqdm(enumerate(models)):
    ens[model] = {}
    for i in tqdm(range(n_ens), leave=True, position=0):
        params[model]['seed'] = im*1000+i
        ebm = EnergyBalanceModel(stochastic_run=True, **params[model][runs[model]])
        ebm.add_forcing(forcing = rfmip_forcing[model], timestep=1)
        ebm.run()
    
        # TODO: method to easily save out results to an object or dict. Glen Harris's version had a nice Results class for this
        ens[model][i] = ebm.__dict__

In [ ]:
fig, ax = pl.subplots(2,4, figsize=(16,9))
for im, model in enumerate(models):
    for i in range(n_ens):
        ax[im//4,im%4].plot(np.arange(1850, 2101), ens[model][i]['temperature'][:,0], color='k', alpha=0.2)
    #for run in branch_points['historical'][model]:
    ax[im//4,im%4].plot(np.arange(1850, 2101), cmip6[model]['tas'].T, color='r', alpha=0.5)
    ax[im//4,im%4].set_xlim(1850,2100)
    ax[im//4,im%4].set_ylim(-1,6)
    ax[im//4,im%4].grid()
    ax[im//4,im%4].set_title(model)

In [ ]:
fig, ax = pl.subplots(2,4, figsize=(16,9))
for im, model in enumerate(models):
    for i in range(n_ens):
        ax[im//4,im%4].plot(np.arange(1850, 2101), ens[model][i]['toa_imbalance'], color='k', alpha=0.2)
    ax[im//4,im%4].plot(np.arange(1850, 2101), cmip6[model]['rndt'].T, color='r', alpha=0.5)
    ax[im//4,im%4].set_xlim(1850,2100)
    ax[im//4,im%4].set_ylim(-2,3)
    ax[im//4,im%4].grid()
    ax[im//4,im%4].set_title(model)

In [ ]:
fig, ax = pl.subplots(2,4, figsize=(16,9))
for im, model in enumerate(models):
    for i in range(n_ens):
        ax[im//4,im%4].plot(np.arange(1850, 2101), ens[model][i]['ocean_heat_content_change'], color='k', alpha=0.2)
    ax[im//4,im%4].plot(np.arange(1850, 2101), np.cumsum(cmip6[model]['rndt'].T, axis=0) * earth_radius**2 * 4 * np.pi * seconds_per_year, color='r', alpha=0.5)
    ax[im//4,im%4].set_xlim(1850,2100)
    ax[im//4,im%4].set_ylim(-5e23, 3.5e24)
    ax[im//4,im%4].grid()
    ax[im//4,im%4].set_title(model)